In [11]:
import cv2
from matplotlib import pyplot as plt

In [ ]:
img = cv2.imread("data/solidYellowLeft.jpg")
#img  = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(hsv, cmap="gray")
plt.title("hsv")
plt.savefig('images/hsv.png', transparent=True)

In [ ]:
mask_yellow = cv2.inRange(hsv, (15,60,20), (25, 255, 255))

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(hsv, cmap="gray")
plt.title("hsv")

plt.subplot(1,2,2)
plt.imshow(mask_yellow, cmap="gray")
plt.title("mask_yellow")

plt.savefig('images/mask_yellow.png', transparent=True)

In [ ]:
mask_white = cv2.inRange(img, (200,200,200), (255, 255, 255))

In [ ]:
plt.figure(figsize=(20,20))

plt.subplot(1,2,1)
plt.imshow(mask_yellow, cmap="gray")
plt.title("mask_yellow")

plt.subplot(1,2,2)
plt.imshow(mask_white, cmap="gray")
plt.title("mask white")

plt.savefig('images/mask_white.png', transparent=True)

In [ ]:
color_mask = cv2.bitwise_or(mask_white, mask_yellow)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(color_mask, cmap="gray")
plt.title("color mask")




In [ ]:
blur = cv2.GaussianBlur(color_mask, (5,5), 0)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(color_mask, cmap="gray")
plt.title("color_mask ")

plt.subplot(1,2,2)
plt.imshow(blur, cmap="gray")
plt.title("blur")

plt.savefig('images/color_mask_blur.png', transparent=True)

In [ ]:
cc = cv2.Canny(blur, 200, 300)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(cc, cmap="gray")
plt.title("c c")

In [12]:
import numpy as np
def roi(img , vec):
    mask = np.zeros_like(img)
    if len(img.shape) == 3:
        mc = (255,) * img.shape[2]
    else:
        mc = 255
    cv2.fillPoly(mask, vec, mc)
    masked = cv2.bitwise_and(img, mask)
    return masked

In [ ]:
height, width = img.shape[0:2]
imshape = img.shape
x_dim = np.shape(img)[1]
y_dim = np.shape(img)[0]
vertices = np.array([[(10,y_dim-50),(x_dim-10,y_dim-50),(x_dim/2+40,y_dim/1.65),(x_dim/2-40,y_dim/1.65)]],
                  dtype = np.int32)
rc = roi(cc, vertices)

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(rc, cmap="gray")
plt.title("r c")

In [ ]:
Lines = cv2.HoughLinesP(rc, rho=2, theta=np.pi/180,threshold=15, 
                        lines=np.array([]), minLineLength=10, maxLineGap=20)



In [ ]:
Lines

In [ ]:
y_size = img.shape[0]
x_size = img.shape[1]
yL_n = 0
xL_n = 0
yR_n = 0
xR_n = 0
yL_n_1 = 5000
xL_n_1 = 0
yR_n_1 = 5000
xR_n_1 = 0
cutoff = 500
for cord in Lines:
    for x1,y1,x2,y2 in cord:
        if x1 < cutoff and x2 < cutoff and y1 > yL_n:
            yL_n = y1
            xL_n = x1
        if x1 < cutoff and x2 < cutoff and y2 > yL_n:
            yL_n = y2
            xL_n = x2
        if x1 < cutoff and x2 < cutoff and y1 < yL_n_1:
            yL_n_1 = y1
            xL_n_1 = x1
        if x1 < cutoff and x2 < cutoff and y2 < yL_n_1:
            yL_n_1 = y2
            xL_n_1 = x2

        if x1 > cutoff and x2 > cutoff and y1 > yR_n:
            yR_n = y1
            xR_n = x1
        if x1 > cutoff and x2 > cutoff and y2 > yR_n:
            yR_n = y2
            xR_n = x2
        if x1 > cutoff and x2 > cutoff and y1 < yR_n_1:
            yR_n_1 = y1
            xR_n_1 = x1
        if x1 > cutoff and x2 > cutoff and y2 < yR_n_1:
            yR_n_1 = y2
            xR_n_1 = x2

m = np.copy(img)
mL = (yL_n - yL_n_1) / (xL_n - xL_n_1)
mR = (yR_n - yR_n_1) / (xR_n - xR_n_1)
bL= (yL_n) - (mL*xL_n)
bR= (yR_n) - (mR*xR_n)
xLB = (y_size - bL) / mL
xRB = (y_size - bR) / mR
Li = np.zeros((img.shape[0] , img.shape[1], 3), dtype=np.uint8)
cv2.line(Li,(int(xLB), y_size),(xL_n_1, yL_n_1),(0,255,255),10)
cv2.line(Li,(int(xRB), y_size),(xR_n_1, yR_n_1),(0,255,255),10)

In [ ]:
m = cv2.addWeighted(m, 0.8, Li, 1, 0)
m = cv2.addWeighted(m, 0.8, Li, 1, 0)

In [ ]:
plt.imshow(m)

In [62]:
cnt =0

In [63]:
def P(img):
    global cnt
    #print(img.shape)
    height, width = img.shape[0:2]
    gray = cv2.cvtColor(img , cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    cc = cv2.Canny(blur, 200, 300)
    height, width = img.shape[0:2]
    imshape = img.shape
    x_dim = np.shape(img)[1]
    y_dim = np.shape(img)[0]
    vertices = np.array([[(10,y_dim-50),(x_dim-10,y_dim-50),(x_dim/2+40,y_dim/1.65),(x_dim/2-40,y_dim/1.65)]],
                  dtype = np.int32)
    rc = roi(cc, vertices)
    Lines = cv2.HoughLinesP(rc, rho=2, theta=np.pi/180,threshold=15, 
                        lines=np.array([]), minLineLength=10, maxLineGap=20)

    y_size = img.shape[0]
    x_size = img.shape[1]
    yL_n = 0
    xL_n = 0
    yR_n = 0
    xR_n = 0
    yL_n_1 = 5000
    xL_n_1 = 0
    yR_n_1 = 5000
    xR_n_1 = 0
    cutoff = 500
    for cord in Lines:
        for x1,y1,x2,y2 in cord:
            if x1 < cutoff and x2 < cutoff and y1 > yL_n:
                yL_n = y1
                xL_n = x1
            if x1 < cutoff and x2 < cutoff and y2 > yL_n:
                yL_n = y2
                xL_n = x2
            if x1 < cutoff and x2 < cutoff and y1 < yL_n_1:
                yL_n_1 = y1
                xL_n_1 = x1
            if x1 < cutoff and x2 < cutoff and y2 < yL_n_1:
                yL_n_1 = y2
                xL_n_1 = x2

            if x1 > cutoff and x2 > cutoff and y1 > yR_n:
                yR_n = y1
                xR_n = x1
            if x1 > cutoff and x2 > cutoff and y2 > yR_n:
                yR_n = y2
                xR_n = x2
            if x1 > cutoff and x2 > cutoff and y1 < yR_n_1:
                yR_n_1 = y1
                xR_n_1 = x1
            if x1 > cutoff and x2 > cutoff and y2 < yR_n_1:
                yR_n_1 = y2
                xR_n_1 = x2
    
    m = np.copy(img)
    try:
        mL = (yL_n - yL_n_1) / (xL_n - xL_n_1)
    except ZeroDivisionError:
        mL = 1 
    mR = (yR_n - yR_n_1) / (xR_n - xR_n_1)
    bL= (yL_n) - (mL*xL_n)
    bR= (yR_n) - (mR*xR_n)
    try:
        xLB = (y_size - bL) / mL
    except ZeroDivisionError:
        xLB = 0
    xRB = (y_size - bR) / mR
    Li = np.zeros((img.shape[0] , img.shape[1], 3), dtype=np.uint8)
    cv2.line(Li,(int(xLB), y_size),(xL_n_1, yL_n_1),(255,255,0),10)
    cv2.line(Li,(int(xRB), y_size),(xR_n_1, yR_n_1),(255,255,0),10)
    
    frames = "frame %d" % cnt
    cv2.putText(Li, frames , (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cnt = cnt + 1


    
    m = cv2.addWeighted(m, 0.8, Li, 1, 0)
    m = cv2.addWeighted(m, 0.8, Li, 1, 0)
    return m
    
    

In [64]:
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/local/bin/ffmpeg"

In [65]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [66]:
in1 = VideoFileClip("data/vids/solidYellowLeft.mp4")
frames = in1.fl_image(P)
frames.write_videofile("data/vids/solidYellowLeft_edge_output.mp4", audio=False)

                                                                                                                                                                 


                                                                                                                                                        
t:  42%|█████████████████████████████████████████████▎                                                              | 286/681 [13:05<00:03, 102.79it/s, now=None]


t:  42%|█████████████████████████████████████████████▊                                                               | 286/681 [06:36<00:04, 97.15it/s, now=None]
                                                                                                                                                                 


                                                                                                                                                        
t:  42%|████████████████████████████████

Moviepy - Building video data/vids/solidYellowLeft_edge_output.mp4.
Moviepy - Writing video data/vids/solidYellowLeft_edge_output.mp4





t:   0%|                                                                                                                       | 0/681 [00:00<?, ?it/s, now=None]

t:   1%|█▏                                                                                                             | 7/681 [00:00<00:09, 69.26it/s, now=None]

t:   5%|█████▉                                                                                                       | 37/681 [00:00<00:03, 201.45it/s, now=None]

t:   9%|█████████▎                                                                                                   | 58/681 [00:00<00:04, 146.14it/s, now=None]

t:  11%|███████████▊                                                                                                 | 74/681 [00:00<00:04, 122.76it/s, now=None]

t:  13%|██████████████                                                                                               | 88/681 [00:00<00:05, 113.72it/s, now=None]

t:  15%|████████████

t:  81%|████████████████████████████████████████████████████████████████████████████████████████▌                    | 553/681 [00:05<00:01, 89.83it/s, now=None]

t:  83%|██████████████████████████████████████████████████████████████████████████████████████████                   | 563/681 [00:05<00:01, 91.43it/s, now=None]

t:  84%|███████████████████████████████████████████████████████████████████████████████████████████▋                 | 573/681 [00:05<00:01, 90.46it/s, now=None]

t:  86%|█████████████████████████████████████████████████████████████████████████████████████████████▊               | 586/681 [00:06<00:00, 96.78it/s, now=None]

t:  88%|███████████████████████████████████████████████████████████████████████████████████████████████▋             | 598/681 [00:06<00:00, 96.30it/s, now=None]

t:  89%|█████████████████████████████████████████████████████████████████████████████████████████████████▍           | 609/681 [00:06<00:00, 99.54it/s, now=None]

t:  91%|██████████████

Moviepy - Done !
Moviepy - video ready data/vids/solidYellowLeft_edge_output.mp4
